In [2]:
import os
import pandas as pd

legacy_data = pd.read_csv("fig2fig_collated_dataframe.csv", index_col=0)

In [3]:
## Consolidating previous variations on SelectFileButton

import traitlets
from ipywidgets import widgets
from IPython.display import display
from tkinter import Tk, filedialog
import os

class SelectFileGeneralButton(widgets.Button):
    """
    Allowed functions: 'askdirectory', 'askopenfilename', 'asksaveasfile'
    Code adapted with minor variation from: https://codereview.stackexchange.com/questions/162920/file-selection-button-for-jupyter-notebook

    """
    def __init__(self, function="askdirectory", label=None):
        super(SelectFileGeneralButton, self).__init__()
        self.add_traits(files=traitlets.traitlets.List())        
        self.icon = "square-o"
        self.style.button_color = "orange"
        self.on_click(self.select_files)
        
        button_label_dict = {'askdirectory': "Select folder",
                             'askopenfilename': "Select file",
                             'asksaveasfile': "Save as..."}
        self._function = function
        
        if type(label) == type(None):
            self.description = button_label_dict[function]
        else:
            self.description = label
        
        
    @staticmethod
    def select_files(b):
        root = Tk()
        root.withdraw()
        root.call("wm", "attributes", ".", "-topmost", True)
        if b._function == "askdirectory":
            b.files = [filedialog.askdirectory()]
            b.description = os.path.split(b.files[0])[-1]

        if b._function == "askopenfilename":
            b.files = [filedialog.askopenfilename()]
            b.description = os.path.split(b.files[0])[-1]

        if b._function == "asksaveasfile":
            files = [('csv Files', "*.csv")] 
            b.files = [filedialog.asksaveasfile(filetypes = files, defaultextension = files)]
            b.description = os.path.split(b.files[0].name)[-1] ## saveasfile returns a io wrapper rather than a string?

        b.icon = "check-square-o"
        b.style.color = "lightgreen"

In [4]:
class SelectFilesTab(object):
    def __init__(selectfiles_self):
        """
        Tab for specifying whether to reload previous data or create new files
        """
        
        import ipywidgets as wg
        import numpy as np
        
        select_previous_files_label = wg.Label(value="Reload previous data")                     
        selectfiles_self.select_previous_files_button = SelectFileGeneralButton(function="askopenfilename", label="csv database")           ## Button to open file explorer for reloading existing dataset
        selectfiles_self.convert_to_csv_button = SelectFileGeneralButton(function="askopenfilename", label="Convert .npy to .csv")
        
        selectfiles_self.select_from_database = wg.Dropdown(options=["No csv selected"])
        selectfiles_self.confirm_dropdown_selection = wg.Button(description="Confirm")
        
        save_new_files_label = wg.Label(value="Create new data")                                 ## Label
        selectfiles_self.save_new_csv_button = SelectFileGeneralButton(function="asksaveasfile")                                    ## Button to open file explorer for creating new file
        
        selectfiles_self.confirm_save_location_button = wg.Button(description="Confirm")                ## Button to confirm selection and set variable name
        selectfiles_self.confirmed_location_label = wg.Label(value="Please select a directory")  ## Instruction
        
        
        ## Arranging widgets for display in 'tab'
        selectfiles_self.vbox = wg.VBox([wg.HBox([select_previous_files_label,
                                                  selectfiles_self.select_previous_files_button,
                                                  selectfiles_self.convert_to_csv_button,
                                                  ]),
                                         wg.HBox([save_new_files_label, selectfiles_self.save_new_csv_button]),
                                         wg.HBox([selectfiles_self.confirm_save_location_button, selectfiles_self.confirmed_location_label]),
                                         wg.HBox([selectfiles_self.select_from_database, selectfiles_self.confirm_dropdown_selection])])
        
    ## Function triggered by clicking 'confirm' button
    ## Trigger function in 'Bibliography' class: uses this function, with save_path string and reload boolean provided by Bibliography
    def reload_values(selectfiles_self, save_path, save_method=None, reload=True):
        import pandas as pd
        import numpy as np
        
        if reload == True:
            selectfiles_self.confirmed_location_label.value = "Reloading values from {}".format(os.path.split(save_path)[-1])
        else:
            selectfiles_self.confirmed_location_label.value = "Saving values to {}".format(os.path.split(save_path)[-1])
            
        ## For reloading from .csv database:
        if save_method == "from_csv":
            selectfiles_self._database = pd.read_csv(save_path, index_col=0)  
            if "concat_label" not in selectfiles_self._database.columns:
                ## From https://stackoverflow.com/a/39291596
                make_concat_labels = selectfiles_self._database[["First author", "Year", "Optional label"]].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
                concat_labels = np.unique(make_concat_labels)
                
                fixed_df = selectfiles_self._database
                fixed_df["concat_label"] = make_concat_labels
                fixed_df.to_csv(save_path)
                                
            else:
                concat_labels = np.unique(selectfiles_self._database["concat_label"])
            
            selectfiles_self.using_dropdown = True
            selectfiles_self.select_from_database.options = list(concat_labels)
            
            
    def reload_from_csv(selectfiles_self):
        ## Used when second confirm button is clicked
        import pandas as pd
        import numpy as np
        import glob
        import os
        
        if selectfiles_self.using_dropdown == True:
            concat_label_value = selectfiles_self.select_from_database.value
            selected_article_df = selectfiles_self._database.loc[selectfiles_self._database["concat_label"]==concat_label_value]
#             print("reload_from_csv, True for selectfiles_self.using_dropdown, sets concat_label_value as: "+concat_label_value)
            
            if len(selectfiles_self.select_from_database.options) > 1:
                print("reloading from multiple - need to create new save path")
                old_versions = len(glob.glob(os.path.join("{}*.csv".format(concat_label_value))))
                if old_versions == 0:
#                     print("No previous versions exist")
                    selected_article_df.to_csv("{}.csv".format(concat_label_value))
                    setattr(selectfiles_self, "save_path",  "{}.csv".format(concat_label_value))
#                     print("Local save_path set to: "+selectfiles_self.save_path)
                else:
#                     print("Previous version exists - appending to prevent overwrite")

                    selected_article_df.to_csv("{}({}).csv".format(concat_label_value, old_versions+1))
                    setattr(selectfiles_self, "save_path", "{}({}).csv".format(concat_label_value, old_versions+1))
#                     print("Local save_path set to: "+selectfiles_self.save_path)

                            
            setattr(selectfiles_self, "_selected_article_df",  selected_article_df)
        
    
            
    ## NB if doing conversion, remember to set _selected_article_df as input

In [5]:
selectfiles_self = SelectFilesTab()
selectfiles_self.vbox

In [6]:
selectfiles_self.save_new_csv_button.files[0].name

IndexError: list index out of range

In [7]:
class ArticleDetailsTab(object):
    def __init__(article_self):
        import ipywidgets as wg
        import numpy as np
        import os
        
        article_self.fields = ["First author", "Year", "Optional label"]
        article_self.entries = {"First author": wg.Text(description="First author", value=""),
                               "Year": wg.IntText(description="Year"),
                                "Optional label": wg.Text(description="Optional label", value=""),
                                "doi": wg.Text(description="doi", value="")}
        
#         article_self.entries = dict([(keys, wg.Text(description=keys, value="")) for keys in article_self.fields])
        article_self.LiSTAR = wg.Checkbox(description="LiSTAR affiliated?")                   ## Originally aimed to track whether LiSTAR results are consistent with SOTA - actually rarely-used
        
        article_self.save_button = wg.Button(description="Save")                             ## Button - TODO set save function!
        article_self.save_label = wg.Label(value="No values saved this session")             ## Label to confirm whether values saved and how recently
        

                
        article_self.vbox = wg.VBox([*article_self.entries.values()]+          ## Text entry boxes for each field
                                    [article_self.LiSTAR]+                                   ## LiSTAR checkbox
                                    [wg.HBox([article_self.save_button, article_self.save_label])])    ## Save button and confirmation label
        
    ## Reload values button triggered by 'confirm' button in selectfiles tab
    ## Function set in outer Bibliography class
    def reload_values(article_self, selected_article_df, save_path, data_from_csv=True):
        if data_from_csv == True:
            article_details_fields = article_self.fields+["doi"]
            reloaded_values = dict([(name, selected_article_df[name].iloc[0]) 
                                             for name in article_self.entries.keys() if name in selected_article_df.columns])
            for keys, values in reloaded_values.items():    ## Appears to do the same thing twice? TODO: Check whether redundant   
                if "LiSTAR_affiliated" in keys:  ## Sets boolean checkbox value (separate from dictionary with author, year, and optional label)
                    article_self.LiSTAR.value = reloaded_values["LiSTAR_affiliated"]
                else:
                    if keys != "Year":
                        if keys in reloaded_values.keys():
                            article_self.entries[keys].value = str(values)
                    else:
                        article_self.entries["Year"].value = int(values)
            
            article_self.save_label.value = "Reloaded values from {}".format(os.path.split(save_path)[-1]) ## Updates confirmation value
        else:
            print("Article details not yet saved")
            
    def save_values(article_self, save_path="test"):
        from datetime import datetime
        import pandas as pd
        
        concat_label = article_self._get_concat_label()
        article_details = dict([(keys, values.value) for keys, values in article_self.entries.items()])
        article_details.update([("concat_label", concat_label)])
        
        article_details.update([("LiSTAR_affiliated", article_self.LiSTAR.value)])
        
        
        ## Testing only
        if save_path == "test":
            save_path = "test_data.csv"
        setattr(article_self, "_output", article_details)
        
        df_to_save = pd.DataFrame(dict([(keys, [values]) for keys, values in article_details.items()]))
        print(save_path)
        df_to_save.to_csv(save_path)
        
        print(article_details.keys())
#         np.save(os.path.join(save_path, "article_details.npy"), article_details, allow_pickle=True)
        save_time = datetime.now()
        article_self.save_label.value = "Values saved at: "+save_time.strftime("%H:%M:%S")
        
    def _get_concat_label(article_self):
        return " ".join((article_self.entries["First author"].value, 
                         str(article_self.entries["Year"].value),
                         article_self.entries["Optional label"].value))
#         return " ".join((values.value for values in article_self.entries.values()))

In [100]:
class _CommonHost(object):
    def __init__(commonhost_self):
        '''
        Subsection of Host tab for saving descriptors common to all entered host types.
        Individual host samples are defined in _HostSample() accordion defined separately to enable common values to be passed to all entered hosts. 
        
        Updated from version 1: Reformatted to fit on laptop screen better for ease of reference
                                Dropdowns for values with category inputs - hence separate definition of different parameter inputs which were all treated as text in version 1
                                Extra fields for measurands and methods for rarely-reported values
        '''
        
        import ipywidgets as wg
        import numpy as np
        import os
        
        ## 02/12/2024 - added sulfur doping and C/X ratio as radiobuttons
        ## 05/01/2025 - added checkboxes for sulfur loading methods      
        
        ## Common parameters dictionary
        ## Note that some of the displayed labels differ from the dictionary keys for ease of display
        commonhost_self.common_dict = {"Carbon type": wg.Text(description="Carbon type", value=None),
                                        "Carbon source": wg.Text(description="Carbon source", value=None),
                                        "Sulfur loading method": wg.Text(description="Other loading method", value=None), ## generic name to enable re-loading of old data without checkboxes
                                        "Sulfur method thermal checkbox": wg.Checkbox(description="Thermal"),
                                        "Sulfur method thermal details": wg.Text(value="155 degC 12 hours"),
                                        "Sulfur method chemical checkbox": wg.Checkbox(description="Chemical "),
                                        "Sulfur method chemical details": wg.Text(value="Na2S2O3"),
                                        "Sulfur method solution checkbox": wg.Checkbox(description="Solution"),
                                        "Sulfur method solution details": wg.Text(value="CS2"),                                       
                                        "Sulfur loading (wt%)": wg.Text(description="Sulfur loading (wt%)", value=None),
                                        "Sulfur loading (mg/ cm2)": wg.Text(description="Areal loading (mg/cm2)", value=None),
                                        "Surface area (cm2/g)": wg.Text(description="Surface area", value=None),
                                        "Pore volume": wg.Text(description="Pore volume", value=None),
                                        "Typical pore widths": wg.Text(description="Typical pore widths", value=None),
                                        "Pore method": wg.Dropdown(description="Pore method", options=["", "BET/ MBET", "DFT", "BJH", "HK"]),
                                        "SSA pore measurand": wg.Dropdown(description="Sample measured", options=["", "Carbon only", "Carbon+sulfur", "Electrode"]),
                                        "Host conductivity": wg.Text(description="Conductivity", value=None),
                                        "Host conductivity units": wg.Text(description="Units", value=None),
                                        "Conductivity method": wg.Dropdown(description="Conductivity method", options=["", "4-point probe", "Pellet", "Other"]),
                                        "Conductivity measurand": wg.Dropdown(description="Sample measured", options=["", "Carbon only", "Carbon+sulfur", "Electrode"]),
                                        "Binder": wg.Text(description="Binder", value=None),
                                        "Binder content (wt%)": wg.Text(description="Content (wt%)", value=None),
                                        "Conductive additive": wg.Text(description="Conductive additive", value=None),
                                        "Conductive additive content (wt%)": wg.Text(description="Content (wt%)", value=None),
                                        "Electrode solvent": wg.Text(description="Electrode solvent"),
                                        "Electrode thickness": wg.Text(description="Electrode thickness"),
                                        "Electrode thickness method": wg.Dropdown(description="Thickness method", options=["", "Micrometer", "Doctor blade", "SEM", "Density"]),
                                      ## DOPANTS AND FUNCTIONAL GROUPS
                                       "Nitrogen content": wg.FloatText(description="Nitrogen"),
                                       "Nitrogen units": wg.RadioButtons(options=["at%", "wt%", "C/N ratio", "Present not stated"]),
                                       "Oxygen content": wg.FloatText(description="Oxygen"),
                                       "Oxygen units": wg.RadioButtons(options=["at%", "wt%", "C/O ratio", "Present not stated"]),
                                       "Sulfur content": wg.FloatText(description="Sulfur"),
                                    "Sulfur units": wg.RadioButtons(options=["at%", "wt%", "C/S ratio", "Present not stated"]),
                                       "Functionalisation method": wg.Dropdown(description="Functionalisation method", options=["", "In precursor", "Surface treatment", "Other"]),
                                       "Other functionalisation method": wg.Text(description="Other functionalisation method"),
                                      }
        
        ## Boolean list of common values
        ## Differentiated by dictionary key, not by label! (for ease of display)
        ## Descriptors that are common to all entered hosts are marked as True
        ## Information is used by 'Update common' button defined in HostTab class to transfer from common fields to sample accordion
        commonhost_self.common_checkbox = dict([(keys, wg.Checkbox(description="", indent=False)) 
                                                for keys, values in commonhost_self.common_dict.items()])
        
        
        
        ## VBox to combine text entry fields/ dropdowns with corresponding Boolean checkboxes
        ## Manually defined so that related fields can be placed in HBoxes next to each other where useful to do so. 
        commonhost_self.common_vbox = wg.VBox([wg.HBox([commonhost_self.common_dict["Carbon type"], commonhost_self.common_checkbox["Carbon type"]]),
                                                        wg.HBox([commonhost_self.common_dict["Carbon source"], commonhost_self.common_checkbox["Carbon source"]]),
                                               ## Sulfur loading method
                                                        wg.Label(value="Sulfur loading methods"),
                                                        wg.HBox([commonhost_self.common_dict["Sulfur method thermal checkbox"],
                                                                 commonhost_self.common_checkbox["Sulfur method thermal checkbox"],
                                                                 commonhost_self.common_dict["Sulfur method thermal details"],
                                                                 commonhost_self.common_checkbox["Sulfur method thermal details"]]),
                                                       wg.HBox([commonhost_self.common_dict["Sulfur method chemical checkbox"],
                                                                 commonhost_self.common_checkbox["Sulfur method chemical checkbox"],
                                                                 commonhost_self.common_dict["Sulfur method chemical details"],
                                                                 commonhost_self.common_checkbox["Sulfur method chemical details"]]),
                                                       wg.HBox([commonhost_self.common_dict["Sulfur method solution checkbox"],
                                                                 commonhost_self.common_checkbox["Sulfur method solution checkbox"],
                                                                 commonhost_self.common_dict["Sulfur method solution details"],
                                                                 commonhost_self.common_checkbox["Sulfur method solution details"]]),
                                                        wg.HBox([commonhost_self.common_dict["Sulfur loading method"], 
                                                                 commonhost_self.common_checkbox["Sulfur loading method"]]),
                                               ##
                                                        wg.HBox([commonhost_self.common_dict["Sulfur loading (wt%)"], commonhost_self.common_checkbox["Sulfur loading (wt%)"],
                                                                commonhost_self.common_dict["Sulfur loading (mg/ cm2)"], commonhost_self.common_checkbox["Sulfur loading (mg/ cm2)"]]),
                                                        wg.HBox([commonhost_self.common_dict["Surface area (cm2/g)"], commonhost_self.common_checkbox["Surface area (cm2/g)"],
                                                                commonhost_self.common_dict["Pore volume"], commonhost_self.common_checkbox["Pore volume"]]),
                                                        wg.HBox([commonhost_self.common_dict["Typical pore widths"], commonhost_self.common_checkbox["Typical pore widths"]]),
                                                        wg.HBox([commonhost_self.common_dict["SSA pore measurand"], commonhost_self.common_checkbox["SSA pore measurand"],
                                                                commonhost_self.common_dict["Pore method"], commonhost_self.common_checkbox["Pore method"]]),
                                                        wg.HBox([commonhost_self.common_dict["Binder"], commonhost_self.common_checkbox["Binder"],
                                                                commonhost_self.common_dict["Binder content (wt%)"], commonhost_self.common_checkbox["Binder content (wt%)"]]),
                                                        wg.HBox([commonhost_self.common_dict["Conductive additive"], commonhost_self.common_checkbox["Conductive additive"],
                                                                commonhost_self.common_dict["Conductive additive content (wt%)"], commonhost_self.common_checkbox["Conductive additive content (wt%)"]]),
                                                        wg.HBox([commonhost_self.common_dict["Electrode solvent"], commonhost_self.common_checkbox["Electrode solvent"]]),
                                                        wg.HBox([commonhost_self.common_dict["Electrode thickness"], commonhost_self.common_checkbox["Electrode thickness"],
                                                                commonhost_self.common_dict["Electrode thickness method"], commonhost_self.common_checkbox["Electrode thickness method"]]),
                                                        ## ADDITION OF DOPANTS AND FUNCTIONAL GROUPS
                                                        wg.Label(value="Dopants and functional groups"),
                                                        wg.HBox([commonhost_self.common_dict["Nitrogen content"], commonhost_self.common_checkbox["Nitrogen content"], commonhost_self.common_dict["Nitrogen units"], commonhost_self.common_checkbox["Nitrogen units"]]),
                                                        wg.HBox([commonhost_self.common_dict["Oxygen content"],  commonhost_self.common_checkbox["Oxygen content"], commonhost_self.common_dict["Oxygen units"], commonhost_self.common_checkbox["Oxygen content"]]),
                                               wg.HBox([commonhost_self.common_dict["Sulfur content"],  commonhost_self.common_checkbox["Sulfur content"], commonhost_self.common_dict["Sulfur units"], commonhost_self.common_checkbox["Sulfur content"]]),
                                                        wg.HBox([commonhost_self.common_dict["Functionalisation method"], commonhost_self.common_checkbox["Functionalisation method"],
                                                                 commonhost_self.common_dict["Other functionalisation method"], commonhost_self.common_checkbox["Other functionalisation method"]])
                                        ])
        ## Display VBox with header
        ## TODO: format header to be more visually apparent for clarity
        commonhost_self.vbox = wg.VBox([wg.Label("Common values")]+[commonhost_self.common_vbox])
        
        
    ## Reload values function triggered by confirming save location in load/ reload tab
    ## Trigger function defined in outer Bibliography class to enable save path to be passed from load/reload tab to host tab
    def reload_values(commonhost_self, host_data_values):
        import pandas as pd
        import numpy as np
        commonhost_fields = [*commonhost_self.common_dict.keys()]

        for name in commonhost_fields:
            if name in host_data_values.columns:
                try:
                    if len(np.unique(host_data_values[name]))==1:
                        commonhost_self.common_dict[name].value = np.unique(host_data_values[name])[0]
                        commonhost_self.common_checkbox[name].value = True
                except:
                    pass

In [101]:
class _HostSample(object):
    def __init__(host_self, test=False):
        '''
        Subsection of Host tab for 
        '''
        import ipywidgets as wg
        import numpy as np
        import os


        
        sulfur_loading_names = {"thermal": wg.Checkbox(description="Thermal"),
                          "chemical": wg.Checkbox(description="Chemical "),
                          "solution": wg.Checkbox(description="Solution")}

        sulfur_loading_details = {"thermal": wg.Text(value="155 deg C 12 hours"),
                                  "chemical": wg.Text(value="Na2S2O3"),
                                  "solution": wg.Text(value="CS2")}

        sulfur_loading_methods = wg.VBox([wg.HBox([sulfur_loading_names[keys], sulfur_loading_details[keys]]) for keys in sulfur_loading_details.keys()]+
                                         [wg.Text(description="Other")])
        
        host_self.sample_dict = {"Host label": wg.Text(description="Host label", value=None),
                                    "Carbon type": wg.Text(description="Carbon type", value=None),
                                    "Carbon source": wg.Text(description="Carbon source", value=None),
                                    "Sulfur loading method": wg.Text(description="Other loading method", value=None), ## generic name to enable re-loading of old data without checkboxes
                                    "Sulfur method thermal checkbox": wg.Checkbox(description="Thermal"),
                                    "Sulfur method thermal details": wg.Text(value="155 degC 12 hours"),
                                    "Sulfur method chemical checkbox": wg.Checkbox(description="Chemical "),
                                    "Sulfur method chemical details": wg.Text(value="Na2S2O3"),
                                    "Sulfur method solution checkbox": wg.Checkbox(description="Solution"),
                                    "Sulfur method solution details": wg.Text(value="CS2"),       
                                    "Sulfur loading (wt%)": wg.Text(description="Sulfur loading (wt%)", value=None),
                                    "Sulfur loading (mg/ cm2)": wg.Text(description="Areal loading (mg/cm2)", value=None),              
                                    "Surface area (cm2/g)": wg.Text(description="Surface area", value=None),
                                    "Pore volume": wg.Text(description="Pore volume", value=None),
                                    "Typical pore widths": wg.Text(description="Typical pore widths", value=None),
                                    "Pore method": wg.Dropdown(description="Pore method", options=["", "BET/ MBET", "DFT", "BJH", "HK"]),
                                    "SSA pore measurand": wg.Dropdown(description="Sample measured", options=["", "Carbon only", "Carbon+sulfur", "Electrode"]),
                                    "Host conductivity": wg.Text(description="Conductivity", value=None),
                                    "Host conductivity units": wg.Text(description="Units", value=None),
                                    "Conductivity method": wg.Dropdown(description="Conductivity method", options=["", "4-point probe", "Pellet", "Other"]),
                                    "Conductivity measurand": wg.Dropdown(description="Sample measured", options=["", "Carbon only", "Carbon+sulfur", "Electrode"]),
                                    "Binder": wg.Text(description="Binder", value=None),
                                    "Binder content (wt%)": wg.Text(description="Content (wt%)", value=None),
                                    "Conductive additive": wg.Text(description="Conductive additive", value=None),
                                    "Conductive additive content (wt%)": wg.Text(description="Content (wt%)", value=None),
                                    "Electrode solvent": wg.Text(description="Electrode solvent"),
                                    "Electrode thickness": wg.Text(description="Electrode thickness"),
                                    "Electrode thickness method": wg.Dropdown(description="Thickness method", 
                                                                              options=["", "Micrometer", "Doctor blade", "SEM", "Density"]),
                                    "Sulfur present in XRD": wg.RadioButtons(description="S$_{8}$ peaks visible in XRD?",
                                                                             options=["No data", "True", "False"],
                                                                             value="No data"),
                                    "Nitrogen content": wg.FloatText(description="Nitrogen"),
                                    "Nitrogen units": wg.RadioButtons(options=["at%", "wt%", "C/N ratio", "Present not stated"]),
                                    "Oxygen content": wg.FloatText(description="Oxygen"),
                                    "Oxygen units": wg.RadioButtons(options=["at%", "wt%", "C/O ratio", "Present not stated"]),
                                    "Sulfur content": wg.FloatText(description="Sulfur"),
                                    "Sulfur units": wg.RadioButtons(options=["at%", "wt%", "C/S ratio", "Present not stated"]),
                                    "Functionalisation method": wg.Dropdown(description="Functionalisation method", 
                                                                            options=["", "In precursor", "Surface treatment", "Other"]),
                                    "Other functionalisation method": wg.Text(description="Other functionalisation method"),}
                
        host_self.vbox = wg.VBox([host_self.sample_dict["Host label"],
                                    host_self.sample_dict["Carbon type"],
                                    host_self.sample_dict["Carbon source"],
                                     ## Sulfur loading method
                                  wg.Label(value="Sulfur loading methods"),
                                    wg.HBox([host_self.sample_dict["Sulfur method thermal checkbox"],
                                             host_self.sample_dict["Sulfur method thermal details"]]),
                                   wg.HBox([host_self.sample_dict["Sulfur method chemical checkbox"],
                                             host_self.sample_dict["Sulfur method chemical details"]]),
                                   wg.HBox([host_self.sample_dict["Sulfur method solution checkbox"],
                                             host_self.sample_dict["Sulfur method solution details"]]),                                 
                                            host_self.sample_dict["Sulfur loading method"],
                                    wg.HBox([host_self.sample_dict["Sulfur loading (wt%)"],
                                                host_self.sample_dict["Sulfur loading (mg/ cm2)"]]),
                                    wg.HBox([host_self.sample_dict["Surface area (cm2/g)"],
                                                host_self.sample_dict["Pore volume"]]),
                                    wg.HBox([host_self.sample_dict["Typical pore widths"]]),
                                    wg.HBox([host_self.sample_dict["SSA pore measurand"],
                                                host_self.sample_dict["Pore method"]]),
                                    wg.HBox([host_self.sample_dict["Binder"],
                                                host_self.sample_dict["Binder content (wt%)"]]),
                                    wg.HBox([host_self.sample_dict["Conductive additive"],
                                                host_self.sample_dict["Conductive additive content (wt%)"]]),
                                                host_self.sample_dict["Electrode solvent"],
                                    wg.HBox([host_self.sample_dict["Electrode thickness"],
                                                host_self.sample_dict["Electrode thickness method"]]),
                                    wg.HBox([host_self.sample_dict["Sulfur present in XRD"]]),
                                    wg.Label(value="Functional groups"),
                                    wg.HBox([host_self.sample_dict["Nitrogen content"], host_self.sample_dict["Nitrogen units"]]),
                                    wg.HBox([host_self.sample_dict["Oxygen content"], host_self.sample_dict["Oxygen units"]]),
                                    wg.HBox([host_self.sample_dict["Sulfur content"], host_self.sample_dict["Sulfur units"]]),
                                    wg.HBox([host_self.sample_dict["Functionalisation method"],
                                             host_self.sample_dict["Other functionalisation method"]])
                                        ])
        if test == True:
            host_self.sample_dict["Host label"].value = "Things"
        
        
    def reload_values(host_self, host_data_values, XRD_data):
        import ipywidgets as wg
        import numpy as np
        
        host_self.host_data_values = host_data_values
        
        
        for keys, values in host_self.sample_dict.items():
            if type(values.value) == type(None):
                print(keys)
            try:
                if keys in host_data_values.keys():
                    if type(values.value) != type(None):
                        values.value = str(host_data_values[keys])
            except:
                pass
                
        if type(XRD_data) != type(None):
            print("Received XRD data OK")
            host_key = str(host_self.sample_dict["Host label"].value)
            if host_self.sample_dict["Host label"].value in XRD_data.keys():
                if type(XRD_data[host_key]) == type(None):
                    host_self.sample_dict["Sulfur present in XRD"].value = "No data"
                elif XRD_data[host_key] == True:
                    host_self.sample_dict["Sulfur present in XRD"].value = "True"
                else:
                    host_self.sample_dict["Sulfur present in XRD"].value = "False"                 

In [102]:
## General addition of host version

class _SampleList_add(object):
    def __init__(sample_self, test=False):
        
#         from CommonHost import _CommonHost
#         from HostSample import _HostSample
        
        import ipywidgets as wg
        import numpy as np
        
        ## Initialise with 1 Host:
        sample_self.sample_list = {"Host 1": _HostSample(test=test)}
        
        add_host_button = wg.Button(description="Add host")
        sample_self.accordion_dict = {"Host 1": wg.VBox([sample_self.sample_list["Host 1"].vbox])}
        
        sample_self.accordion = wg.Accordion([*sample_self.accordion_dict.values()])
        sample_self.rename_accordion()
        
        sample_self.remove_button_dict = {"Host 1": wg.VBox([])}
        
    def add_new_host_general(sample_self, reload_values=None):
        ## Check maximum name:
        import re
        import numpy as np
        import ipywidgets as wg
        
        last_host = np.max([int(re.findall("\d+", keys)[0]) for keys in sample_self.sample_list.keys()])
        
        ## Add a new Host object to the sample_dict
        sample_self.sample_list.update([("Host {}".format(last_host+1), _HostSample())])
        sample_self.remove_button_dict.update([("Host {}".format(last_host+1), 
                                                wg.Button(description="Remove host {}".format(last_host+1)))])
                
        def on_remove_button_clicked(b):
            sample_self.remove_host_button(b)
        
        sample_self.remove_button_dict["Host {}".format(last_host+1)].on_click(on_remove_button_clicked)
        
        ## If reloading, add the new values, otherwise skip
        if type(reload_values) != type(None): 
            sample_self.sample_list["Host {}".format(last_host+1)].reload_values(reload_values, XRD_data=None)
            
        else:
            sample_self.accordion_dict = dict([(list_keys, wg.VBox([sample_self.remove_button_dict[list_keys],
                                                                        sample_self.sample_list[list_keys].vbox])) 
                                                   for list_keys, list_values in sample_self.sample_list.items()])
            
           
        sample_self.accordion.children = [*sample_self.accordion_dict.values()] ## Update children rather than accordion to update in place
        sample_self.rename_accordion()
        
        
    def rename_accordion(sample_self):
        for title, (index, _) in zip([*sample_self.accordion_dict.keys()], enumerate(sample_self.accordion.children)):
            sample_self.accordion.set_title(index, "{}".format(title))
            
    def remove_host_button(sample_self, b):
        import re
        host_id = int(re.findall("\d+", b.description)[0])
        host_name = "Host {}".format(host_id)
        if host_name in sample_self.sample_list.keys():
            sample_self.sample_list.pop(host_name)
            sample_self.accordion_dict.pop(host_name)
                
        sample_self.accordion.children = [*sample_self.accordion_dict.values()] ## Update children rather than accordion to update in place
        sample_self.rename_accordion()
        
    def reload_values(sample_self, reload_values, XRD_data):
#         print("reload_clicked")
        import ipywidgets as wg

        for keys, values in reload_values.items():
            if keys in sample_self.sample_list.keys():
                sample_self.sample_list[keys].reload_values(values, XRD_data)
            else:
                sample_self.add_new_host_general(reload_values=values)
                sample_self.accordion_dict = dict([(list_keys, wg.VBox([sample_self.remove_button_dict[list_keys],
                                                                        sample_self.sample_list[list_keys].vbox])) 
                                                   for list_keys, list_values in sample_self.sample_list.items()])

        sample_self.accordion.children = [*sample_self.accordion_dict.values()] ## Update children rather than accordion to update in place
        sample_self.rename_accordion()
        
    def save_values(sample_self, save_path, concat_label):
        import numpy as np
        import os
        import pandas as pd
        
#         import warnings
#         warnings.filterwarnings(action="ignore", message="SettingWithCopyWarning") ## TODO work out why the copy warning appears and fix properly!

        existing_data = pd.read_csv(save_path, index_col=0)
        host_data = existing_data.copy()
        sample_self._host_data = host_data
        
        host_values_arr = np.array([[value.value for value in sample_values.sample_dict.values()] for sample_key, sample_values in sample_self.sample_list.items()]).T
        host_keys = [*[*sample_self.sample_list.values()][0].sample_dict.keys()]
        host_dict = dict([(key, list(host_values_arr[nkey])) for nkey, key in enumerate(host_keys)])
               
        def cont_keyword(df, keyword, column="Host label", case=True):
            return df[df[column].astype(str).str.contains(keyword, case)]

        for sample_keys, sample_values in sample_self.sample_list.items():
            host_label = sample_values.sample_dict["Host label"].value
            
            if host_keys[0] not in host_data.columns: ## If this is the first host to be entered and new columns need to be made:
                print("first entry to be added")
                for host_label, host_value in host_dict.items():
                    for idx in host_data.index:
                        
                        host_data.loc[idx, host_label] = host_value[idx]
#                         print(idx, host_value[idx])
            else:
                current_entry = cont_keyword(host_data, host_label)

                if len(current_entry) == 0: ## If the entry is not in the dataframe:
                    current_length = max(host_data.index)
                    new_idx = current_length+1

                    for column in host_data.columns:
                        if column not in host_keys:
                            host_data.loc[new_idx, column] = host_data.loc[current_length, column]
                        else:
                            host_data.loc[new_idx, column] = sample_values.sample_dict[column].value

                if len(cont_keyword(host_data, host_label)) == 1: ## If the entry is in the dataframe:
                    for column in host_keys:
                        host_data.loc[current_entry.index, column] = sample_values.sample_dict[column].value

        for name in host_data["Host label"]:
#             print(name)
            if name not in host_dict["Host label"]: ## Checks dataframe for extraneous names and removes them
#                 print(host_dict["Host label"])
                index_to_drop = host_data.loc[host_data["Host label"]==name].index[0]
                host_data.drop(index_to_drop, inplace=True)

        host_data.reset_index(drop=True, inplace=True)
        
        host_data.to_csv(save_path)

In [103]:
class HostTab(object):
    def __init__(hosttab_self):
        
        import ipywidgets as wg
        import numpy as np
        import os
        
        hosttab_self.commonhost_self = _CommonHost()
        hosttab_self.sample_self = _SampleList_add()
        
        hosttab_self.save_button = wg.Button(description="Save")        
        hosttab_self.update_common_button = wg.Button(description="Update common")
        hosttab_self.add_new_host_button = wg.Button(description="Add new host")
        hosttab_self.save_confirm_label = wg.Label(value="No new values saved this session")
        
        def on_update_common_clicked(b):
            for sample_keys, sample_values in hosttab_self.sample_self.sample_list.items():
                for host_keys, host_values in sample_values.sample_dict.items():
                    if host_keys in hosttab_self.commonhost_self.common_checkbox.keys():
                        if hosttab_self.commonhost_self.common_checkbox[host_keys].value == True:
                            host_values.value = hosttab_self.commonhost_self.common_dict[host_keys].value
                            
        def on_add_new_host_clicked(b):
            hosttab_self.sample_self.add_new_host_general(reload_values=None)

        hosttab_self.update_common_button.on_click(on_update_common_clicked)
        hosttab_self.add_new_host_button.on_click(on_add_new_host_clicked)
        
        hosttab_self.vbox = wg.VBox([hosttab_self.commonhost_self.vbox,
                                     wg.HBox([hosttab_self.save_button, hosttab_self.save_confirm_label]),
                                     hosttab_self.add_new_host_button,
                                     hosttab_self.update_common_button,
                                     hosttab_self.sample_self.accordion])
        
    def reload_values(hosttab_self, selected_article_df, save_path):
        host_details_fields = list(hosttab_self.sample_self.sample_list["Host 1"].sample_dict.keys())
              
        
        ## Making container
        host_details_from_csv = dict([("Host {}".format(n+1), 
                  dict([(field, None) for field in host_details_fields])) for n in range(len(selected_article_df))])
        
        hosttab_self.commonhost_self.reload_values(selected_article_df)

        
        for n in range(len(selected_article_df)):
            for field in host_details_fields:
                if field in selected_article_df.columns: ## Some extra fields in new fig2fig version
                    host_details_from_csv["Host {}".format(n+1)][field] = selected_article_df[field].iloc[n]
                
            hosttab_self.sample_self.reload_values(reload_values=host_details_from_csv, XRD_data=None)

    def save_values(hosttab_self):
        from datetime import datetime
        save_time = datetime.now()        
        # Actual save function performed elsewhere
        hosttab_self.save_confirm_label.value = "Values saved at: "+save_time.strftime("%H:%M:%S")
        

In [104]:
hosttab_self = HostTab()
hosttab_self.vbox

In [ ]:
class ElectrochemistryTab(object):
    def __init__(electrochem_self):
        import ipywidgets as wg
        
        electrochem_self.entries = {"Electrolyte salt": wg.Text(description="Electrolyte salt", value="LiTFSI"),
                                    "Salt content": wg.FloatText(description="Salt content", value=1),
                                    "salt_units": wg.RadioButtons(options=["M", "wt%"], value="M"),
                                    "LiNO3 content": wg.FloatText(description="LiNO$_{3}$ content"),
                                    "LiNO3_units": wg.RadioButtons(options=["M", "wt%"], value="M"),
                                    "Electrolyte solvent": wg.Text(description="Electrolyte solvent", value="1:1 DOL DME"),
                                    "Electrolyte volume (uL)": wg.FloatText(description="Electrolyte volume $\mu$L"),
                                    "E/S ratio (uL/g)": wg.FloatText(description="E/S ratio ($\mu$L/g)"),
                                    "Separator type": wg.Text(description="Separator type"),
                                    "Cell type": wg.Text(description="Cell type"),
                                    "Maximum voltage": wg.FloatText(description="Maximum voltage"),
                                    "Minimum voltage": wg.FloatText(description="Minimum voltage"),
                                    "Common voltage range": wg.Checkbox(description="Same voltage range for all rates?",
                                                                        value=True)}
        
        electrochem_self.save_button = wg.Button(description="Save")
        electrochem_self.save_label = wg.Label(value="No values saved yet")
        
        electrochem_self.vbox = wg.VBox([electrochem_self.entries["Electrolyte salt"],
                                         wg.HBox([electrochem_self.entries["Salt content"], electrochem_self.entries["salt_units"]]),
                                         wg.HBox([electrochem_self.entries["LiNO3 content"], electrochem_self.entries["LiNO3_units"]]),
                                         electrochem_self.entries["Electrolyte solvent"],
                                         wg.HBox([electrochem_self.entries["Electrolyte volume (uL)"],
                                                  electrochem_self.entries["E/S ratio (uL/g)"]]),
                                         electrochem_self.entries["Separator type"],
                                         electrochem_self.entries["Cell type"],
                                         electrochem_self.entries["Common voltage range"],
                                         wg.HBox([electrochem_self.entries["Maximum voltage"],
                                                  electrochem_self.entries["Minimum voltage"]]),
                                         wg.HBox([electrochem_self.save_button, electrochem_self.save_label])])
        
    def save_values(electrochem_self, save_path, concat_label):
        from datetime import datetime
        
        electrochem_values = dict([(keys, values.value) for keys, values in electrochem_self.entries.items()])

        existing_data = pd.read_csv(save_path, index_col=0)
        electrochem_data = existing_data.copy()
        
        for keys, values in electrochem_values.items(): 
            electrochem_data.loc[electrochem_data["concat_label"]==concat_label, keys] = values
            
        electrochem_data.to_csv(save_path)
                
        now = datetime.now()        
        electrochem_self.save_label.value = "Values saved at: "+now.strftime("%H:%M:%S")
        
    def reload_values(electrochem_self, save_path, concat_label):
        existing_data = pd.read_csv(save_path, index_col=0)

        for keys, values in electrochem_self.entries.items():
            if keys in existing_data.columns:
                ## TODO - tidy up if possible - unclear why this is so picky!!
                if type(electrochem_self.entries[keys].value) == str:
                    electrochem_self.entries[keys].value = str(existing_data.loc[existing_data["concat_label"]==concat_label][keys].iloc[0])
                elif type(electrochem_self.entries[keys].value) == float:
                    electrochem_self.entries[keys].value = float(existing_data.loc[existing_data["concat_label"]==concat_label][keys].iloc[0])
                elif type(electrochem_self.entries[keys].value) == bool:
                    electrochem_self.entries[keys].value = bool(existing_data.loc[existing_data["concat_label"]==concat_label][keys].iloc[0])       

In [ ]:
class CharacterisationTab(object):
    def __init__(char_self):
        
        import ipywidgets as wg
        import numpy as np
        import os
        
        
        char_self.physical_methods = ["SEM", "XPS", "Raman", "EDX", "TEM",  ]
        char_self.physical_measurands = ["Host", "S/C composite", "Electrode", "In-situ", "Post mortem"]
        
        char_self.physical_grid = dict([(method,
               dict([(measurand, wg.Checkbox(description="", indent=False, layout=wg.Layout(width="50pt"))) for measurand in char_self.physical_measurands]))
               for method in char_self.physical_methods])
        
        
        char_self.ec_methods = {"CV (single rate)": {"CV (single rate)": wg.Checkbox(description="CV (single rate)"), 
                                                     "First cycle": wg.Checkbox(description="1st cycle"),
                                                     "Subsequent cycles": wg.Checkbox(description="Subsequent cycles")},
                                "CV (multi rate)": {"CV (multi rate)": wg.Checkbox(description="CV (multi rate)"), 
                                                    "Linear peak current vs. v1/2":  wg.Checkbox(description="Linear peak current vs. $v^{0.5}$")}, 
                                "EIS": {"EIS": wg.Checkbox(description="EIS data"),
                                        "Rest time": wg.Checkbox(description="Rest period"),
                                        "Min/ Max frequency": wg.Checkbox(description="Frequency range", value=True),
                                        "ECM": wg.Checkbox(description="Equiv. circuit model"),
                                        "KK/ error": wg.Checkbox(description="Kramers Kroenig")}
             }
        
        char_self.phys_hbox = wg.HBox([wg.VBox([wg.Label(meas_name) for meas_name in [""]+char_self.physical_measurands])]+
                                 [wg.VBox([wg.Label(value=method_keys)]+[*method_values.values()])
                                  for method_keys, method_values in char_self.physical_grid.items()])
        
        char_self.ec_vbox = wg.VBox([wg.HBox([char_self.ec_methods["CV (single rate)"]["CV (single rate)"],
                  char_self.ec_methods["CV (single rate)"]["First cycle"],
                  char_self.ec_methods["CV (single rate)"]["Subsequent cycles"]]),
         wg.HBox([char_self.ec_methods["CV (multi rate)"]["CV (multi rate)"], 
                  char_self.ec_methods["CV (multi rate)"]["Linear peak current vs. v1/2"]]),
         wg.VBox([char_self.ec_methods["EIS"]["EIS"],
                  wg.HBox([char_self.ec_methods["EIS"]["Rest time"], char_self.ec_methods["EIS"]["Min/ Max frequency"]]),
                  wg.HBox([char_self.ec_methods["EIS"]["ECM"], char_self.ec_methods["EIS"]["KK/ error"]])])
         ])
        
        char_self.save_button = wg.Button(description="Save")
        char_self.save_confirm_label = wg.Label(value="No values saved this session")
        
        char_self.vbox = wg.VBox([wg.Label("Physical characterisation"),
                                 char_self.phys_hbox,
                                 wg.Label("Electrochemical characterisation"),
                                 char_self.ec_vbox,
                                         wg.HBox([char_self.save_button, char_self.save_confirm_label])
                                        ])
        
    def save_values(char_self, save_path, concat_label):
        physical_char_values = dict([(" ".join((method, measurand)), char_self.physical_grid[method][measurand].value) for method in char_self.physical_methods for measurand in char_self.physical_measurands])
        
        ec_char_values = dict([(name_key, dict([(detail_key, detail_value.value) for detail_key, detail_value in name_values.items()])) 
                             for name_key, name_values in char_self.ec_methods.items()])
        
        ec_char_values = {**ec_char_values["CV (single rate)"], **ec_char_values["CV (multi rate)"], **ec_char_values["EIS"]} ## probably imperfect, but so far fixed list of keys!
        char_values_merged = {**physical_char_values, **ec_char_values}
        
        existing_data = pd.read_csv(save_path, index_col=0)
        from copy import deepcopy

        ec_char_data = deepcopy(existing_data)

        for keys, values in char_values_merged.items():
            ec_char_data.loc[ec_char_data["concat_label"] == concat_label, keys] = values
            
        ec_char_data.to_csv(save_path)
        
        from datetime import datetime
        
        now = datetime.now()
        char_self.save_confirm_label.value = "Values saved at: "+now.strftime("%H:%M:%S")
        
    def reload_values(char_self, save_path, concat_label):
        existing_data = pd.read_csv(save_path, index_col=0)
        ## For physical characterisation:
        for method_key, method_value in char_self.physical_grid.items():
            for measurand_key, measurand_value in method_value.items():
                if " ".join((method_key, measurand_key)) in existing_data.columns:                  
                    bool_result = existing_data.loc[existing_data["concat_label"]==concat_label][" ".join((method_key, measurand_key))].iloc[0]
                    char_self.physical_grid[method_key][measurand_key].value = bool(bool_result)
                
        ## For electrochemical characterisation:
        for method_key, method_value in char_self.ec_methods.items():
            for measurand_key, measurand_value in method_value.items():
                if measurand_key in existing_data.columns:
                    bool_result = existing_data.loc[existing_data["concat_label"]==concat_label][measurand_key].iloc[0] ## Currently unique measurand keys within inner dicts, may need to change
                    char_self.ec_methods[method_key][measurand_key].value = bool(bool_result)

In [ ]:
class Bibliography(object):
    def __init__(self):
        import ipywidgets as wg
        import numpy as np
        
        
        self._select_files_tab = SelectFilesTab()
        self._article_details_tab = ArticleDetailsTab()
        self._host_tab = HostTab()
        self._electrochemistry_tab = ElectrochemistryTab()
        self._characterisation_tab = CharacterisationTab()
        
        def on_confirm_save_location_clicked(b):
            ## If loading from a previous .csv database:
            if len(self._select_files_tab.select_previous_files_button.files) > 0:
                self.save_path=self._select_files_tab.select_previous_files_button.files[0]
                self._select_files_tab.reload_values(save_path=self.save_path, save_method="from_csv")
                
            ## If converting old directory with .npy files to .csv:
            if len(self._select_files_tab.convert_to_csv_button.files) > 0:
                print("Conversion method in use")
              
#             if len(self._select_files_tab.select_from_database.options) > 0:
#                 self.save_path = self._select_files_tab.select_from_database.value
#                 print("On confirm save location clicked: "+self.save_path)
# #                 else:
# #                     self.save_path = self._select_files_tab.select_from_database.options[0].name
                    
                self._select_files_tab.confirmed_location_label.value = "Saving new data"                


        def on_confirm_dropdown_clicked(b):
            if self._select_files_tab.using_dropdown == True:

                self._select_files_tab.reload_from_csv()
                if "save_path" in vars(self._select_files_tab):
                    self.save_path = self._select_files_tab.save_path
                
                self._article_details_tab.reload_values(selected_article_df=self._select_files_tab._selected_article_df,
                                                        save_path = self.save_path)
                self._concat_label = self._article_details_tab._get_concat_label()
                
                self._host_tab.reload_values(selected_article_df=self._select_files_tab._selected_article_df,
                                             save_path=self.save_path)
                self._characterisation_tab.reload_values(save_path=self.save_path, concat_label=self._concat_label)

                
        self._select_files_tab.confirm_save_location_button.on_click(on_confirm_save_location_clicked)
        self._select_files_tab.confirm_dropdown_selection.on_click(on_confirm_dropdown_clicked)
        
        def on_save_article_clicked(b):
            self._article_details_tab.save_values(save_path=self.save_path)
            self._concat_label = self._article_details_tab._get_concat_label()
            print(self._concat_label)
        self._article_details_tab.save_button.on_click(on_save_article_clicked)
        
        def on_save_host_values_clicked(b):
            
            self._host_tab.sample_self.save_values(save_path=self.save_path, concat_label=self._concat_label)
            self._host_tab.save_values()
        self._host_tab.save_button.on_click(on_save_host_values_clicked)
        
        def on_save_electrochem_clicked(b):
            self._electrochemistry_tab.save_values(save_path=self.save_path, concat_label=self._concat_label)
        self._electrochemistry_tab.save_button.on_click(on_save_electrochem_clicked)    
        
        def on_save_characterisation_clicked(b):
            self._characterisation_tab.save_values(save_path=self.save_path, concat_label=self._concat_label)
        self._characterisation_tab.save_button.on_click(on_save_characterisation_clicked)
        
        self.tab = wg.Tab([self._select_files_tab.vbox,
                           self._article_details_tab.vbox,
                           self._host_tab.vbox,
                           self._electrochemistry_tab.vbox,
                           self._characterisation_tab.vbox
                          ]) 
        
        tab_names = ["Save and reload",
                     "Article details", 
                     "Host", 
                     "Electrochemistry",
                     "Characterisation"
                    ]
        for n, names in enumerate(tab_names):
            self.tab.set_title(n, names)

In [ ]:
self = Bibliography()
self.tab

In [ ]:
pd.read_csv(self.save_path, index_col=0)

In [ ]:
existing_data = pd.read_csv("fig2fig_collated_dataframe.csv", index_col=0)

In [ ]:
self._article_details_tab.save